### Text-Input Processing and Tokenization

In [1]:
from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
prompt = "A sunset over the mountains"
text_inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

c:\Users\techi\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\techi\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\techi\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as 

### Text Encoding

In [2]:
from transformers import CLIPTextModel

text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
text_embeddings = text_encoder(**text_inputs).pooler_output

c:\Users\techi\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\techi\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

### Latent Space Initialization

In [ ]:
import torch
latent_vector = torch.randn(1, 4, 64, 64)  # Example latent space

### Diffusion Model (UNet)

In [ ]:
from diffusers import UNet2DConditionModel

unet_model = UNet2DConditionModel.from_pretrained("stabilityai/stable-diffusion-2")
refined_latent = unet_model(latent_vector, text_embeddings)

### Scheduler for Denoising

In [ ]:
from diffusers import PNDMScheduler

scheduler = PNDMScheduler.from_pretrained("stabilityai/stable-diffusion-2")
for _ in range(num_steps):
    latent_vector = scheduler.step(latent_vector, refined_latent)

### Image Decoding(VAE)

In [ ]:
from diffusers import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2")
generated_image = vae.decode(latent_vector).sample

### Post-processing

In [ ]:
from PIL import Image
import numpy as np

image_array = (generated_image[0].permute(1, 2, 0).numpy() * 255).astype("uint8")
final_image = Image.fromarray(image_array)
final_image.save("output_image.png")

### Pipeline Integration

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2").to("cuda")
generated_image = pipeline("A sunset over the mountains").images[0]
generated_image.show()